In [ ]:
import copy

import prody
from rdkit import Chem
from rdkit.Chem import Draw
from prody.proteins.functions import showProtein, view3D

from rgroup import rep2D, draw3D, merge_R_group, fix_receptor, optimise_in_receptor, RGroups, build_molecules, sort_conformers

In [ ]:
# load the complex with the ligand
sys = prody.parsePDB('5NEV.pdb1')
rec = sys.select('not (nucleic or hetatm) and not water')
mol = sys.select('resname 72L')

prody.writePDB('rec.pdb', rec)
# prody.writePDB('lig.pdb', mol)

In [ ]:
!wget 'https://models.rcsb.org/v1/5nev/ligand?auth_seq_id=301&label_asym_id=E&encoding=sdf&filename=5nev_E_72L.sdf' -O lig.sdf
!obabel lig.sdf -O ligh.sdf -h -p 7
# ^manually modify the files if need be

In [ ]:
# load the ligand
# rdkit doesn't do a good job with protonation, 
#init_mol = Chem.MolFromPDBFile('lig.pdb', removeHs=False)
#init_mol = Chem.AddHs(init_mol, addCoords=True)

init_mol = Chem.SDMolSupplier('ligh.sdf', removeHs=False)[0]

In [ ]:
# select the atom to be replaced with the R group
Draw.MolToImage(rep2D(init_mol))

In [ ]:
# look up r groups and select
RGroups

In [ ]:
# build a list of molecules
mols = build_molecules(init_mol, [47], RGroups)

In [ ]:
mols[0].draw3D()

In [ ]:
Draw.MolsToGridImage([rep2D(mol, idx=False) for mol in mols] )

In [ ]:
# take the first molecule
rmol = mols[0]
Draw.MolToImage(rep2D(rmol, idx=False))

In [ ]:
toxicity = rmol.toxicity()
toxicity

In [ ]:
# visualise the template
# draw3D(rmol.template)

In [ ]:
rmol.draw3D()

In [ ]:
rmol.generate_conformers(num_conf=30, minimum_conf_rms=0.5)

In [ ]:
rmol.draw3Dconfs()

In [ ]:
rmol.GetNumConformers()

In [ ]:
# fix the receptor file 
fix_receptor("rec.pdb", "rec_h.pdb")
# load back into prody
rec_fix = prody.parsePDB("rec_h.pdb")

In [ ]:
# filter by the protein clashes
view = view3D(rec_fix)
rmol.draw3Dconfs(view)

In [ ]:
import py3Dmol
view = py3Dmol.view(width=300, height=300, viewergrid=(1, 1))

for conf in rmol.GetConformers():
    mb = Chem.MolToMolBlock(rmol, confId=conf.GetId())
    view.addModel(mb, "lig")

view.setStyle({'stick':{}})
view.zoomTo()

In [ ]:
rmol.removeConfsClashingWithProdyProt(rec_fix)

In [ ]:
view = view3D(rec_fix)
rmol.draw3Dconfs(view)

In [ ]:
opt_mol, energies = optimise_in_receptor(
     ligand=rmol, 
     receptor_file="rec_h.pdb", 
     ligand_force_field="openff", 
     use_ani=True,
     sigma_scale_factor=0.8,
     relative_permittivity=4
 )

In [ ]:
type(opt_mol)
opt_mol.draw3Dconfs()

In [ ]:
opt_mol.to_file("optimised_corrds.pdb")

In [ ]:
print(energies)

In [ ]:
final_mol, final_energies = sort_conformers(ligand=opt_mol, energies=energies, energy_range=5)

In [ ]:
final_mol.to_file("best_conformers.pdb")

In [ ]:
print(final_energies)